In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

ON_KAGGLE = Path("/kaggle").exists()
if ON_KAGGLE:
    sys.path.insert(0, "/kaggle/input/kaggle-utils")
    %run /kaggle/input/kaggle-utils/stat_funcs.py
    train_df = pd.read_csv("/kaggle/input/playground-series-s6e1/train.csv")
    test_df  = pd.read_csv("/kaggle/input/playground-series-s6e1/test.csv")
else:
    %run ../../utils/stat_funcs.py
    train_df = pd.read_csv("../../data-raw/train.csv")
    test_df  = pd.read_csv("../../data-raw/test.csv")

print("custom functions are now available in the notebook namespace!")
print("Libraries loaded successfully!")

Exception: File `'/kaggle/input/kaggle-utils/stat_funcs.py'` not found.

## Data Exploration

Target column: `exam_score`

In [ ]:
train_df.head(10)

Missing values

In [ ]:
train_df.isnull().sum()

Data Types

In [ ]:
train_df.dtypes

In [ ]:
for col in ["gender", "course", "internet_access", "sleep_quality", "study_method", "facility_rating", "exam_difficulty"]:
  train_df[col] = train_df[col].astype('category')

train_df.dtypes

Statistical Summary

In [ ]:
train_df.describe()

Missing Values

In [ ]:
print(train_df.isnull().sum())

## What do we got?

### Averages

- 20.5 year old students,
- who studies for 4 hours, and
- are in class ~71.9% of the time.
- They sleep about 7 hours,
- and score about ~62.5%.

### Extremes

- We have students who are as young as 17 years and as old as 24 years.
- Some students study for only 0.08 hours?! while others study for 8 hours.
- We have some who are almost in all classes and others who are present only ~40% of the time.
- Lowest exam score is 19.5% and the highest is 100%.

### Other
- No missing values in any variables

## Exam Score Analysis

In [ ]:
## Target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(train_df['exam_score'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Distribution of Exam Scores', fontsize=14)
axes[0].set_xlabel('Exam Score')
axes[0].set_ylabel('Frequency')

# Box plot
axes[1].boxplot(train_df['exam_score'], vert=True)
axes[1].set_title('Box Plot of Exam Scores', fontsize=14)
axes[1].set_ylabel('Exam Score')

plt.tight_layout()
plt.show()

In [ ]:
print(f"Mean:   {train_df['exam_score'].mean():.2f}")
print(f"Median: {train_df['exam_score'].median():.2f}")
print(f"Std:    {train_df['exam_score'].std():.2f}")

## Categorical Feature Analysis (univariate analysis)

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Gender vs Exam Scores

In [ ]:
fig, ax = plt.subplots(figsize=(16, 14))

sns.boxplot(data=train_df, x='gender', y='exam_score', ax=ax)
ax.set_title('Exam Score by Gender', fontsize=12)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
summary = (
    train_df.groupby("gender")["exam_score"]
            .agg(
                n="count",
                mean="mean",
                median="median",
                std="std",
                q1=lambda s: s.quantile(0.25),
                q3=lambda s: s.quantile(0.75)
            )
)

print(summary)

One-way ANOVA via Ordinary Least Squares

In [ ]:
df = train_df.dropna(subset=["exam_score", "gender"]).copy()

model = smf.ols("exam_score ~ C(gender)", data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

Eta-squared = SS_between / SS_total

In [ ]:
ss_between = anova_table.loc["C(gender)", "sum_sq"]
ss_resid = anova_table.loc["Residual", "sum_sq"]
ss_total = ss_between + ss_resid
eta_sq = ss_between / ss_total

In [ ]:
print(anova_table)
print(f"\nEta-squared (η^2): {eta_sq:.6f}")

- **ANOVA result:** (F(`2`, `629,997`)=`55.43`), (`p`~ `8.5e-25`).
  - With a sample this large, **at least one gender group mean differs from another** in a statistically detectable way.
- **Effect size (η^2 = 0.000176):** This is the key interpretation for “how big.”
  - eta^2 is the **proportion of total variation in exam scores explained by gender**.
  - eta^2 = 0.000176 ~ 0.0176%

So **gender explains about 0.018% of the variance** in exam scores - essentially none in practical terms.

Even though the p-value is tiny (because we have ~630k rows), **gender is not meaningfully related to exam score** in terms of how much it helps explain or predict scores. The group means may differ by a small amount, but the difference is **very small relative to the overall spread of scores**.

### Exam score by Course

In [ ]:
fig, ax = plt.subplots(figsize=(16, 14))

sns.boxplot(data=train_df, x='course', y='exam_score', ax=ax)
ax.set_title('Exam Score by Course', fontsize=12)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
summary = (
    train_df.groupby("course")["exam_score"]
            .agg(
                n="count",
                mean="mean",
                median="median",
                std="std",
                q1=lambda s: s.quantile(0.25),
                q3=lambda s: s.quantile(0.75)
            )
)

print(summary)

In [ ]:
df = train_df.dropna(subset=["exam_score", "course"]).copy()

model = smf.ols("exam_score ~ C(course)", data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

ss_between = anova_table.loc["C(course)", "sum_sq"]
ss_resid = anova_table.loc["Residual", "sum_sq"]
ss_total = ss_between + ss_resid
eta_sq = ss_between / ss_total

In [ ]:
print(anova_table)
print(f"\nEta-squared (η^2): {eta_sq:.6f}")

- **ANOVA result:** (F(`6`, `629,993`)=`32.52`), (`p`~ `2.1e-39`).
  - With a sample this large, **at least one course group mean differs from another** in a statistically detectable way.
- **Effect size (η^2 = 0.000310):**
  - eta^2 is the **proportion of total variation in exam scores explained by course**.
  - eta^2 = 0.000310 ~ 0.0310%

So **course explains about 0.031% of the variance** in exam scores - essentially none in practical terms.

Even though the p-value is tiny (because we have ~630k rows), **course is not meaningfully related to exam score** in terms of how much it helps explain or predict scores. Any differences in course-level means exist, but they are **very small relative to the overall spread of scores**. Given the negligible η², any statistically significant pairwise differences between specific course categories are expected to be very small; interpret with effect sizes (e.g., Hedges’ g) rather than p-values alone.

### Exam score by Internet Access

In [ ]:
fig, ax = plt.subplots(figsize=(16, 14))

sns.boxplot(data=train_df, x='internet_access', y='exam_score', ax=ax)
ax.set_title('Exam Score by Internet Access', fontsize=12)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
summary = (
    train_df.groupby("internet_access")["exam_score"]
            .agg(
                n="count",
                mean="mean",
                median="median",
                std="std",
                q1=lambda s: s.quantile(0.25),
                q3=lambda s: s.quantile(0.75)
            )
)

print(summary)

In [ ]:
internet_access_yes = train_df[train_df["internet_access"]=="yes"]["exam_score"].to_numpy()
internet_access_no = train_df[train_df["internet_access"]=="no"]["exam_score"].to_numpy()

print(f"Cohen's D: {cohend(internet_access_yes, internet_access_no):.6f}")

#### Cohen's D interpretation

**Intuition**
- d = 0: the group averages are the same.
- d = 1: the averages are one standard deviation apart (that’s pretty noticeable).
- d = 0.5: half a standard deviation apart (moderate).
- d = 0.2: small difference.

**Common rule-of-thumb (very rough)**
- 0.2 = small
- 0.5 = medium
- 0.8 = large

With **exam score** as the outcome:

- Mean (no internet) = **62.48**
- Mean (yes internet) = **62.51**
- Difference = **~0.03 points**
- **Cohen’s d = 0.0016** = **basically zero effect**

Students with and without internet access scored **about the same** on the exam.

### Exam Score vs Sleep Quality

In [ ]:
fig, ax = plt.subplots(figsize=(16, 14))

sns.boxplot(data=train_df, x='sleep_quality', y='exam_score', ax=ax)
ax.set_title('Exam Score by Sleep Quality', fontsize=12)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
summary = (
    train_df.groupby("sleep_quality")["exam_score"]
            .agg(
                n="count",
                mean="mean",
                median="median",
                std="std",
                q1=lambda s: s.quantile(0.25),
                q3=lambda s: s.quantile(0.75)
            )
)

print(summary)

### Exam Score vs study method

In [ ]:
fig, ax = plt.subplots(figsize=(16, 14))

sns.boxplot(data=train_df, x='study_method', y='exam_score', ax=ax)
ax.set_title('Exam Score by Study Method', fontsize=12)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
summary = (
    train_df.groupby("study_method")["exam_score"]
            .agg(
                n="count",
                mean="mean",
                median="median",
                std="std",
                q1=lambda s: s.quantile(0.25),
                q3=lambda s: s.quantile(0.75)
            )
)

print(summary)

In [ ]:
df = train_df.dropna(subset=["exam_score", "study_method"]).copy()

model = smf.ols("exam_score ~ C(study_method)", data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

ss_between = anova_table.loc["C(study_method)", "sum_sq"]
ss_resid = anova_table.loc["Residual", "sum_sq"]
ss_total = ss_between + ss_resid
eta_sq = ss_between / ss_total

In [ ]:
print(anova_table)
print(f"\nEta-squared (η²): {eta_sq:.6f}")

- **ANOVA result:** (F(`4`, `629,995`)=`8304.29`), (`p` ~ `0.0`).
  - With a sample this large, **at least one study method group mean differs from another** in a statistically detectable way.
- **Effect size (η² = 0.050085):**
  - eta^2 is the **proportion of total variation in exam scores explained by study_method**.
  - eta^2 = 0.050085 ~ 5.01%

So **study_method explains about 5.0% of the variance** in exam scores - a **meaningful** relationship (small-to-moderate).

Unlike gender/course (where η² was near zero), here the differences are also obvious in the group means:
- coaching: **69.27**
- mixed: **65.10**
- group study: **60.53**
- online videos: **59.73**
- self-study: **57.70**

That’s roughly an **~11.6 point spread** from coaching to self-study, which is large relative to the within-group SDs (~18–19). This is why the effect size is non-negligible.

### Exam SCore by facility rating

In [ ]:
fig, ax = plt.subplots(figsize=(16, 14))

sns.boxplot(data=train_df, x='facility_rating', y='exam_score', ax=ax)
ax.set_title('Exam Score by Facility Rating', fontsize=12)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
summary = (
    train_df.groupby("facility_rating")["exam_score"]
            .agg(
                n="count",
                mean="mean",
                median="median",
                std="std",
                q1=lambda s: s.quantile(0.25),
                q3=lambda s: s.quantile(0.75)
            )
)

print(summary)

In [ ]:
df = train_df.dropna(subset=["exam_score", "facility_rating"]).copy()

model = smf.ols("exam_score ~ C(facility_rating)", data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

ss_between = anova_table.loc["C(facility_rating)", "sum_sq"]
ss_resid = anova_table.loc["Residual", "sum_sq"]
ss_total = ss_between + ss_resid
eta_sq = ss_between / ss_total

In [ ]:
print(anova_table)
print(f"\nEta-squared (η²): {eta_sq:.6f}")

- **ANOVA result:** (F(`2`, `629,997`)=`11664.97`), (`p` ~ `0.0`).
  - With a sample this large, **at least one facility_rating group mean differs from another** in a statistically detectable way.
- **Effect size (η² = 0.035709):**
  - eta^2 is the **proportion of total variation in exam scores explained by facility_rating**.
  - eta^2 = 0.035709 ~ 3.57%

So **facility_rating explains about 3.6% of the variance** in exam scores - a **meaningful** relationship (small-to-moderate).

The group means show clear practical separation:
- **high:** 66.71  
- **medium:** 63.03  
- **low:** 57.95  

That’s an **~8.75 point** difference between high vs low, which is sizeable relative to the within-group SDs (~18.5–18.6), consistent with the non-negligible η².

### Exam Score by Exam Difficulty

In [ ]:
fig, ax = plt.subplots(figsize=(16, 14))

sns.boxplot(data=train_df, x='exam_difficulty', y='exam_score', ax=ax)
ax.set_title('Exam Score by Exam Difficulty', fontsize=12)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
summary = (
    train_df.groupby("exam_difficulty")["exam_score"]
            .agg(
                n="count",
                mean="mean",
                median="median",
                std="std",
                q1=lambda s: s.quantile(0.25),
                q3=lambda s: s.quantile(0.75)
            )
)

print(summary)

In [ ]:
df = train_df.dropna(subset=["exam_score", "exam_difficulty"]).copy()

model = smf.ols("exam_score ~ C(exam_difficulty)", data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

ss_between = anova_table.loc["C(exam_difficulty)", "sum_sq"]
ss_resid = anova_table.loc["Residual", "sum_sq"]
ss_total = ss_between + ss_resid
eta_sq = ss_between / ss_total

In [ ]:
print(anova_table)
print(f"\nEta-squared (η²): {eta_sq:.6f}")

- **ANOVA result:** (F(`2`, `629,997`)=`29.74`), (`p`~ `1.21e-13`).
  - With a sample this large, **at least one exam_difficulty group mean differs from another** in a statistically detectable way.
- **Effect size (η² = 0.000094):**
  - eta^2 is the **proportion of total variation in exam scores explained by exam_difficulty**.
  - eta^2 = 0.000094 ~ 0.0094%

So **exam_difficulty explains about 0.009% of the variance** in exam scores - essentially none in practical terms.

The group means are nearly identical:
- **easy:** 62.21  
- **moderate:** 62.61  
- **hard:** 62.67  

The largest gap (hard vs easy) is only **~0.46 points**, which is tiny relative to the within-group SDs (~19). That’s why the p-value is very small (huge N), but the effect size is negligible.

## Numerical Feature Analysis

| Column           | Data Type |
|------------------|-----------|
| age              | int64     | 
| study_hours      | float64   | 
| class_attendance | float64   | 
| sleep_hours      | float64   | 
| exam_score       | float64   | 

In [ ]:
num_cols = ['age', 'study_hours', 'class_attendance', 'sleep_hours', 'exam_score']

plt.figure(figsize=(10, 8))
correlation = train_df[num_cols].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0, fmt='.3f', linewidths=0.5)

plt.title('Correlation Heatmap', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
corr_string = correlation.round(3)
print(corr_string.to_string())

### Correlation summary (Pearson r)

Correlations with exam_score:
- `study_hours`: r = 0.762  (strong positive association)
- `class_attendance`: r = 0.361  (moderate positive association)
- `sleep_hours`: r = 0.167  (weak positive association)
- `age`: r = 0.010  (essentially no association)

Other notable relationships among predictors:
- `study_hours` vs `class_attendance`: r = 0.088 (very weak positive)
- `study_hours` vs `sleep_hours`: r = 0.042 (very weak positive)
- `class_attendance` vs `sleep_hours`: r = 0.029 (very weak positive)
- `age` has near-zero correlation with the other variables (|r| ~ 0.006-0.011)

Exam scores are most strongly associated with `study_hours`, with smaller (but still positive) associations for `class_attendance` and `sleep_hours`. Age appears unrelated to exam performance in this dataset. Also, the predictors are not strongly correlated with each other, so multicollinearity is unlikely to be a major concern for a basic regression model.

## Overall EDA Summary

### Dataset overview 

The training data contains **630,000 rows** and **13 columns**, with the target variable **`exam_score`** (continuous, 0-100 scale). An **`id`** column is a unique identifier (sequential from 0 to 629,999) and is not inherently predictive. The `id` column would need to be removed during feature engineering.

- **Data quality:**
  - **No missing values** across any features or the target.
  - Data types are clean and appropriate after casting:
    - **Numeric:** `age`, `study_hours`, `class_attendance`, `sleep_hours`, `exam_score`
    - **Categorical:** `gender`, `course`, `internet_access`, `sleep_quality`, `study_method`, `facility_rating`, `exam_difficulty`
- **Typical student profile (central tendency):**
  - **Age:** ~**20.55** years (median **21**, range **17-24**)
  - **Study hours:** mean ~**4.00** (median **4.00**, range **0.08-7.91**)
  - **Class attendance (%):** mean ~**71.99** (median **72.6**, range **40.6-99.4**)
  - **Sleep hours:** mean ~**7.07** (median **7.1**, range **4.1-9.9**)
  - **Exam score:** mean ~**62.51** (median **62.6**, range **19.6-100**)
- **Spread / variability:**
  - `exam_score` has **substantial variability** (std ~**18.92**), suggesting meaningful separation between low- and high-performing students.
  - `class_attendance` also varies widely (std ~**17.43**), while `age` is comparatively tight (std ~**2.26**).
- **Notable extremes & sanity checks:**
  - Very low study time values exist (down to **0.08 hours**), and attendance can be as low as **~40%**, which may represent legitimately low-engagement students rather than data issues (since missingness is zero and ranges look plausible).
  - Scores span almost the entire possible scale (**~20 to 100**), indicating no obvious clipping problems.
- **Model-readiness implications:**
  - This is a **mixed-type regression problem** with several categorical predictors that will require **encoding** (one-hot or target/ordinal encoding depending on the feature).


## 2026-01-21

I'm a Chris Deotte fan, and [his EDA](https://www.kaggle.com/code/cdeotte/basic-eda-lots-of-linear-relationships) says:
- We can convert `age` to a categorical due to its low cardinality
- We need to start caring about ordinal classes
- We can see many nearly uniform linear distribution
- Many features have a linear relationship with `exam_score`
- And importantly,
  >All features appear to be predictive of target